## Signature Verification using Siamese Networks

#### work flow
input - 2 images and a label  
get embeddings from a basic model (EffitientNet overfits)  
calculate similarity - how close these 2 vectors are in space  
The label (1 or 0) supervises this learning:
If two images are labeled as similar (1) but embeddings are far → loss is high → update weights.
If labeled as different (0) but embeddings are close → loss is high → update weights.  
higher similarity -> Genuine, lower -> forged

In [1]:
import tensorflow as tf
import os

2025-07-02 18:15:46.479740: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751480146.665296      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751480146.718789      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
path = '/kaggle/input/signature-verification-dataset/sign_data/train'
ls = os.listdir(path)
print('training data directories',len(ls))

training data directories 128


#### Preprocess & Pair the signatures


input format: ( (img1, img2), label)  
label = 1: Two genuine signatures from the same person.  
label = 0: One genuine and one forged signature from the same person.  

In [3]:
import random
from PIL import Image
import numpy as np

def load_signature_pairs(data_dir):
    pairs = []
    labels = []
    
    users = sorted([name for name in os.listdir(data_dir) if '_' not in name])
    
    for user in users:
        genuine_dir = os.path.join(data_dir, user)
        forg_dir = genuine_dir + '_forg'
        
        genuine_imgs = os.listdir(genuine_dir)
        forgery_imgs = os.listdir(forg_dir)
        
        num_genuine = len(genuine_imgs)
        
        # Random positive pairs (genuine vs different genuine)
        random_indices = [
            random.choice([j for j in range(num_genuine) if j != i])
            for i in range(num_genuine)
        ]
        
        for i in range(num_genuine):
            img1 = os.path.join(genuine_dir, genuine_imgs[i])
            img2 = os.path.join(genuine_dir, genuine_imgs[random_indices[i]])
            pairs.append((img1, img2))
            labels.append(1)

        # Negative pairs (genuine vs forgery)
        for i in range(min(len(genuine_imgs), len(forgery_imgs))):
            img1 = os.path.join(genuine_dir, genuine_imgs[i])
            img2 = os.path.join(forg_dir, forgery_imgs[i])
            pairs.append((img1, img2))
            labels.append(0)
    
    return pairs, labels


In [4]:
path = '/kaggle/input/signature-verification-dataset/sign_data/train'
pairs, labels = load_signature_pairs(path)
print(len(pairs), len(labels))

1606 1606


### backbone

caluculate embeddings for each image to compare similarity (how close they are in space)  
Input: 224×224 grayscale image  
Output: 256-dimensional embedding for input image.

In [5]:
from tensorflow.keras import layers, Model
# from tensorflow.keras.applications import EfficientNetB0

IMG_SIZE = 224
# custom layer for L2 normalization
class L2Normalization(tf.keras.layers.Layer):
    def call(self, inputs):
        return tf.nn.l2_normalize(inputs, axis=1)

def build_backbone():
    # # Load EfficientNet WITHOUT pretrained weights
    # base_model = EfficientNetB0(weights=None, include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 1))
    # base_model.trainable = True  # Allow training the backbone
    # x = layers.GlobalAveragePooling2D()(base_model.output)  # Flatten spatial features
    # x = layers.Dense(256, activation='relu')(x)  # Trainable embedding layer
    # x = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)  # Normalize embeddings
    # return Model(base_model.input, x, name="EfficientNetBackbone")
    
    inputs = tf.keras.Input(shape=(224, 224, 1))
    x = layers.Conv2D(64, 3, activation='relu')(inputs) # basic feature extraction
    x = layers.MaxPooling2D()(x) # reduces spatial size
    x = layers.Conv2D(128, 3, activation='relu')(x) # d eeper pattern detection
    x = layers.GlobalAveragePooling2D()(x) # flattens spatial info into a single vector
    x = layers.Dense(256, activation='relu')(x) # embedding layer
    # x = layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x) # scales output to unit length (for similarity comparison)
    # x = tf.nn.l2_normalize(x, axis=1)
    x = L2Normalization()(x)
    return Model(inputs, x, name="SimpleCNNBackbone")


Feature Extraction:
Pass both images through the backbone (shared weights) to get embeddings.  
Similarity Calculation:
Use L1 distance (absolute difference) between the two embeddings.  
Output Layer:
sigmoid activation gives a similarity score (between 0 and 1).  


### siamese neural network

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, Model

def build_siamese_network(backbone):
    input_1 = layers.Input(shape=(224, 224, 1), name="image_1")
    input_2 = layers.Input(shape=(224, 224, 1), name="image_2")

    # Extract embeddings from both images
    embed_1 = backbone(input_1)
    embed_2 = backbone(input_2)

    # Distance layer (L1 distance)
    distance = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([embed_1, embed_2])

    # Final classification head
    output = layers.Dense(1, activation='sigmoid')(distance)

    model = Model(inputs=[input_1, input_2], outputs=output, name="SiameseNetwork")
    return model


In [7]:
backbone = build_backbone()
siamese_model = build_siamese_network(backbone)
siamese_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
siamese_model.summary()


I0000 00:00:1751480163.844130      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "SiameseNetwork"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_1 (InputLayer)      │ (None, 224, 224, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ image_2 (InputLayer)      │ (None, 224, 224, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ SimpleCNNBackbone         │ (None, 256)            │        107,520 │ image_1[0][0],         │
│ (Functional)              │                        │                │ image_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 256)            │              0 │ SimpleCNNBackbone[0][… │
│                           │                        │                │ SimpleCNNBackbone[1][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 1)              │            257 │ lambda[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 107,777 (421.00 KB)

 Trainable params: 107,777 (421.00 KB)

 Non-trainable params: 0 (0.00 B)

input image -> grayscale, [224, 224] px, normalized

In [8]:
def preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=1)  # grayscale
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    return image  # shape: (224, 224, 1)

In [9]:
def make_tf_dataset(pairs, labels, batch_size=32, shuffle=True):
    path_ds = tf.data.Dataset.from_tensor_slices((pairs, labels))

    def load_images(pair, label):
        img1 = preprocess_image(pair[0])
        img2 = preprocess_image(pair[1])
        return (img1, img2), label

    dataset = path_ds.map(load_images, num_parallel_calls=tf.data.AUTOTUNE)
    # num_parallel_calls=tf.data.AUTOTUNE lets TensorFlow load multiple images in parallel = faster.
    dataset = dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

In [10]:
train_dataset = make_tf_dataset(pairs, labels, batch_size=32)
print(len(train_dataset))

51


In [11]:
siamese_model.fit(train_dataset, epochs=10)

Epoch 1/10


I0000 00:00:1751480171.743390      57 service.cc:148] XLA service 0x7afcbc046b20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751480171.744233      57 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1751480172.122511      57 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/51 ━━━━━━━━━━━━━━━━━━━━ 9:02 11s/step - accuracy: 0.5625 - loss: 0.6932

I0000 00:00:1751480175.988989      57 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


51/51 ━━━━━━━━━━━━━━━━━━━━ 17s 132ms/step - accuracy: 0.6920 - loss: 0.6784
Epoch 2/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.8247 - loss: 0.5891
Epoch 3/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.8876 - loss: 0.5203
Epoch 4/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.8966 - loss: 0.4962
Epoch 5/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.9081 - loss: 0.4733
Epoch 6/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.9156 - loss: 0.4506
Epoch 7/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 71ms/step - accuracy: 0.8960 - loss: 0.4325
Epoch 8/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.9145 - loss: 0.4264
Epoch 9/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.9088 - loss: 0.4008
Epoch 10/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.9090 - loss: 0.4010


In [12]:
# Load test data
path = '/kaggle/input/signature-verification-dataset/sign_data/test'
pairs, labels = load_signature_pairs(path)

print("Total samples:", len(pairs))

test_dataset = make_tf_dataset(pairs, labels, batch_size=32)
print("Batches:", tf.data.experimental.cardinality(test_dataset).numpy())

# Evaluate model
loss, accuracy = siamese_model.evaluate(test_dataset)
print(f"\nTest Accuracy: {accuracy:.4f}")

Total samples: 476
Batches: 15
15/15 ━━━━━━━━━━━━━━━━━━━━ 5s 102ms/step - accuracy: 0.9702 - loss: 0.3212

Test Accuracy: 0.9769


In [13]:
img_path1 = '/kaggle/input/signature-verification-dataset/sign_data/train/017_forg/01_0107017.PNG'
img_path2 = '/kaggle/input/signature-verification-dataset/sign_data/train/017/01_017.png'
def predict_similarity(model, img_path1, img_path2):
    img1 = preprocess_image(img_path1)
    img2 = preprocess_image(img_path2)

    # Add batch dimension: (1, 224, 224, 1)
    img1 = tf.expand_dims(img1, axis=0)
    img2 = tf.expand_dims(img2, axis=0)

    prediction = model.predict([img1, img2])[0][0]  # sigmoid output

    print(f"Similarity Score: {prediction:.4f}")
    if prediction >= 0.5:
        print("Genuine")
    else:
        print("Forged ")

In [14]:
predict_similarity(siamese_model, img_path1, img_path2)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 502ms/step
Similarity Score: 0.0276
Forged 


save the model

In [15]:
siamese_model.save("siamese_signature.keras")
# siamese_model.save('siamese_signature.h5')

In [16]:
# import shutil

# shutil.make_archive("siamese_signature_model", 'zip', "siamese_signature_model")